# Importing needed packages

In [1]:
import math
import pandas
import sqlalchemy
print(pandas.__version__)

import geopandas as gpd
import folium
#from folium.plugins import MarkerCluster # for clustering the markers
import plotly.express as px
import plotly.graph_objects as go

# 
pandas.set_option('max_columns', None)

1.3.4


# Define initial variables and necessary funcions
...which stay constant over the use of this Notebook.

In [2]:
# Path to sql query files
SQL_PATH = r"/home/eouser/Documents/code/Windatlas/windatlas/mastr/sqlCommands/"
SQL_DATA = r"loadMastrWind.sql"

SQL_DATA_PATH = SQL_PATH + SQL_DATA

# Parameters to create a connection to the MaStR-postgreSQL DB
CONN_PARAM_DICT = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

In [3]:
def build_postgres_conn_string (param:dict) -> str:
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict) -> sqlalchemy.engine.base.Engine:
    conString = build_postgres_conn_string(param)
    engine = sqlalchemy.create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict) -> pandas.DataFrame:
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pandas.read_sql(script, engine)

    return df

# Loading and preparing Data
First we will load Mastr data from the database with the set connection string.

In [4]:
dfMastrWind = read_postgres_from_queryfile(SQL_DATA_PATH, CONN_PARAM_DICT)

dfMastrWind['SpezifischeLeistung'] = (dfMastrWind.Nettonennleistung * 1000) / ((math.pi * dfMastrWind.Rotordurchmesser**2) / 4)
dfMastrWind["Anlagenzahl"] = 1

dfMastrWind.columns

Index(['EinheitMastrNummer', 'DatumLetzteAktualisierung',
       'LokationMaStRNummer', 'NetzbetreiberpruefungStatus',
       'NetzbetreiberpruefungDatum', 'AnlagenbetreiberMastrNummer', 'Land',
       'Bundesland', 'Landkreis', 'Gemeinde', 'Gemeindeschluessel',
       'Postleitzahl', 'Gemarkung', 'FlurFlurstuecknummern',
       'StrasseNichtGefunden', 'Hausnummer_nv', 'HausnummerNichtGefunden',
       'Ort', 'Laengengrad', 'Breitengrad', 'Registrierungsdatum',
       'Inbetriebnahmedatum', 'EinheitSystemstatus', 'EinheitBetriebsstatus',
       'NichtVorhandenInMigriertenEinheiten', 'DatumDesBetreiberwechsels',
       'DatumRegistrierungDesBetreiberwechsels', 'NameStromerzeugungseinheit',
       'Weic_nv', 'Kraftwerksnummer_nv', 'Energietraeger', 'Bruttoleistung',
       'Nettonennleistung', 'AnschlussAnHoechstOderHochSpannung',
       'FernsteuerbarkeitNb', 'FernsteuerbarkeitDv', 'FernsteuerbarkeitDr',
       'Einspeisungsart', 'GenMastrNummer', 'Lage', 'Hersteller',
       'Technolog

If we want to filter offshore wind turbine, there seams to be no column in the Mastr data which allows us to classifies into on and offshore. There are but two columns which define rather a wind turbine is set in the "Ostsee" or "Nordsee" cluster. Lets look at them, how we could use them to filter offshore wind farms:

In [5]:
#dfMastrWind.loc[:,("Typenbezeichnung","ClusterOstsee","ClusterNordsee")].head(n=20)

# alternativly to calling the `.head()` method of the dataframe, you cann also specivy the raws in the `.loc[]` or `.iloc[]` method:
# iloc stands for index location
#dfMastrWind.iloc[270:290,-15:-10]
# .loc stands for location and is primarily label based, but may also be used with a boolean array. 
# Carefull! Selecting raws by integers as done below, only works if the index contains the given integers. Else you will get an error.
dfMastrWind.loc[270:290,("Typenbezeichnung","ClusterOstsee","ClusterNordsee",'SpezifischeLeistung')] 


,Typenbezeichnung,ClusterOstsee,ClusterNordsee,SpezifischeLeistung
270,E115,NaN,NaN,285.291995
271,None,NaN,NaN,NaN
272,E-82 E2,NaN,NaN,435.522152
273,SVT 7.0 154,NaN,1553.0,394.599032
274,Tacke TW 1.5s,NaN,NaN,384.258200
275,E82,NaN,NaN,378.714915
276,E-40/6.44,NaN,NaN,394.599032
277,E-115 EP3 E3,NaN,NaN,399.408793
278,E-53,NaN,NaN,362.617172
279,E-115,NaN,NaN,285.291995


It seams that only wind turbines which are located at either of the to seas have a valid `float()` value at the corrisponding column. Otherwise a `NaN` value is given. Therefore we can filter the dataframe by raws which have valid values at the `("ClusterOstsee","ClusterNordsee")` columns:

In [6]:
dfMastrWindOffshore = dfMastrWind.dropna(subset=("ClusterOstsee", "ClusterNordsee"), how="all")
dfMastrWindOffshore.loc[:,("Typenbezeichnung","ClusterOstsee", "ClusterNordsee")]

,Typenbezeichnung,ClusterOstsee,ClusterNordsee
7,AD5-116,NaN,1553.0
51,6.2M126,NaN,1548.0
102,6.2M126,NaN,1548.0
120,6.2M126,NaN,1548.0
144,SWT-3.6-120 V3,NaN,1547.0
...,...,...,...
32664,Siemens 6.0-154 PB-HWRT,1540.0,NaN
32691,ADWEN AD5-135,1540.0,NaN
32708,Haliade 150-6MW,NaN,1547.0
32760,SVT 7.0 154,NaN,1553.0


Lets vizualize the offshore wind turbines to interactivly scrolle through them using plotly and the integrated mapbox:

In [7]:
fig = go.Figure(px.scatter_mapbox(dfMastrWindOffshore, 
            lat="Breitengrad", 
            lon="Laengengrad", 
            #animation_frame = 'Inbetriebnahmedatum',
            color="Nettonennleistung", #"SpezifischeLeistung",
            #size="Nettonennleistung",
            hover_name="EegMaStRNummer", 
            hover_data=["Inbetriebnahmedatum","Hersteller","Typenbezeichnung", "Nettonennleistung"],
            color_continuous_scale="jet", 
            zoom=4,
            height=600,
            mapbox_style="open-street-map"))
            

fig.update_layout(margin = {'l':0, 'r':0, 'b':0, 't':0})

fig.show()

As we can see, we have an outlier turbine located in Great Britain. Lets remove it and run the cell above again:

In [8]:
dfMastrWindOffshore = dfMastrWindOffshore.loc[dfMastrWindOffshore.Laengengrad > 4]

### Starting the geo processing

As seen in the list of columns above, we do not have any column naming the associated offshore wind park to the wind turbine. Therefore, we are going to import a `.shp`-file which includes the polygons of all offshore wind parks. We will do so, by using the `geopandas` package, which gives us GIS functionality inside python. 

In order to combine both the wind park polygons and the wind turbine point data, we will need to transform the `dfMastrWindOffshore` pandas dataframe into a geopandas dataframe. Thereby we will pass the Leangengrad and Breitengrad as valid coordinate sources.

Finally we need to set a coordinate system in the mastr geopandas dataframe. We will get it from the the wind park geodataframe:

In [9]:
# Convert pandas df to geopandas with Längen and Breitengrad to coordinates
offshore_geo = gpd.GeoDataFrame(
    dfMastrWindOffshore, 
    geometry=gpd.points_from_xy(dfMastrWindOffshore.Laengengrad, dfMastrWindOffshore.Breitengrad))

# import wind parks from .shp file
shp_Path = r"/home/eouser/Documents/code/Windatlas/windatlas/mastr/geoData/contis_offshorewindfarms.shp"
offshore_parks = gpd.read_file(shp_Path, encoding = 'unicode_escape', engine ='python')
offshore_parks = offshore_parks.rename(columns={"name_": "Windparkname"})

# set crs (coordinatesystem) of points to the one of parks
offshore_geo = offshore_geo.set_crs(offshore_parks.crs)
offshore_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Before using the wind park data, we still need to clean them up a little. `Trianel Windpark Borkum ` has multiple polygons as can be seen bellow:

In [10]:
offshore_parks['Windparkname'].value_counts().sort_values(ascending=False).head()

Trianel Windpark Borkum    2
Merkur Offshore            1
DanTysk                    1
BARD Offshore 1            1
Sandbank                   1
Name: Windparkname, dtype: int64

Therefore we will combine the polygons and add additional information, like the area and the boundary length to each polygon. Further do we need to buffer each polygon, since some wind turbines are placed slightly out of the associated polygon. To make these calculations, we need to transform the coordinate system to one, which allows us to calculate in meters (epsg 3763). After finishing the calculations we need to switch back to the prior crs.

In [14]:
# combine/dissolve wind park polygons of the same park to single polygons
offshore_parks_diss = offshore_parks.dissolve(by='Windparkname', aggfunc='first', as_index=False)

# set crs for calculations in meter in order to calculate geometric parameters of the polygons
offshore_parks_diss = offshore_parks_diss.to_crs(epsg=3763)
# recalculate area and boundary
offshore_parks_diss["Fläche qkm"] = (offshore_parks_diss.geometry.area / 1000000).round(2)
offshore_parks_diss["Umfang km"] = (offshore_parks_diss.geometry.boundary.length / 1000).round(2)
# buffer boundary to catch wind turbines on the edge of windpark polygons
offshore_parks_diss["geometry"] = offshore_parks_diss.geometry.buffer(20)
# reset crs
offshore_parks_diss = offshore_parks_diss.to_crs(epsg=4326)

offshore_parks_diss.head(n=3)

,Windparkname,geometry,featureid,featurespe,status,featuretyp,uuid,Fläche qkm,Umfang km,Sea,E-Charts,ROP site
0,ARCADIS Ost 1,"POLYGON ((13.61149 54.81949, 13.61149 54.81949...",404963,PilotPhase,Approved,Offshore_Windfarms,fcc7affe-9505-4a9a-9d0d-ffd3db0578a6,26.32,27.53,Baltic Sea,None,EO 4
1,Albatros,"POLYGON ((6.24827 54.46601, 6.24825 54.46600, ...",191913,PilotPhase,inUse,Offshore_Windfarms,d21f299b-07d8-4fcb-8055-1049344631b6,10.76,15.12,North Sea,Ja,EN 08
2,Amrumbank West,"POLYGON ((7.64070 54.53883, 7.64071 54.53885, ...",129323,PilotPhase,inUse,Offshore_Windfarms,897101b9-296c-4ba0-8790-1c2cee90c8d3,30.96,25.33,North Sea,Ja,EN 04


Now we can display the different wind parks and wind turbines. For that we will be using the by geopandas recomanded `folium` package, which is build on the more known `leaflet` package. In comparison to plotly and mapbox, folium is a little more complex in how to initially use it, since plotlys scatter_mapbox is more a "all in one function" solution. Folium gives us the oportunity to include multiple layers of features and to configure them with a great level of detail. 

We will set the center for the initial map at the center of the city Kiel and define additional visual parameters for dispalying the polygons and points.

In [15]:
coord_Kiel = {"y":54.32932119717097,
        "x":10.147986691105206}

map = folium.Map(location = [coord_Kiel["y"], coord_Kiel["x"]], tiles='OpenStreetMap' , zoom_start = 7)

# Plotting windpark polygons
for _, row in offshore_parks_diss.iterrows():
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.00001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})

    popup_text = "Park: {}<br> Fläche qkm: {}<br> Umfang km: {}"
    popup_text = popup_text.format(row["Windparkname"],
                                   row["Fläche qkm"],
                                   row["Umfang km"])
    folium.Popup(popup_text).add_to(geo_j)
    geo_j.add_to(map)

# Plotting wind turbines
#marker_cluster = MarkerCluster().add_to(map)
for _, row in offshore_geo.iterrows():

    # Adding popup text, so clicking each point shows details about each home
    popup_text = "Nettonennleistung: {}<br> Latitude: {}<br> Longitude: {}"
    popup_text = popup_text.format(row["Nettonennleistung"],
                                   row["Breitengrad"],
                                   row["Laengengrad"])

    # Changing the color based on buckets of Nettonennleistung
    if row["Nettonennleistung"] < 4000:
        color = "#85CB33" #green
    elif row["Nettonennleistung"] >= 4000 and row["Nettonennleistung"] < 7000:
        color = "#F9B700" #yellow
    else:
        color = "#E01A4F" #hot pink
    
    # Adding each home to the map
    folium.CircleMarker([row['Breitengrad'], row['Laengengrad']],
                        radius=1, #(row["Nettonennleistung"]/1000000),
                        fill=True,
                        color=color,
                        popup=popup_text).add_to(map)

map

We can combine the information of the two dataframes and add the correct Windparkname to each wind turbine in the mastr dataset. The resulting geopandas dataframe will be transformed back into a pandas dataframe.

Since the `("ClusterNordsee","ClusterOstsee")` columns are filled with `int()` values which to not correspond to any information we could use, we will switch them to either 1 for the wind turbine being in that cluster or 0 if it is not in it.

Finally we group the entire wind turbines by the `Windparkname` and apply different aggregation functions to columns of our interest. Since `Typbezeichnungen` are string values of the wind turbine type, we can not simply aggregate numericly without losing the key information. Therefore, we apply a count function to all unique `Typbezeichnungen` for each `Windparkname`and take the most common wind turbine as the representing one for the entire park.

In [13]:
# inner spatial Join for points with Parks to add park information to points
offshore_geo_parks = offshore_geo.sjoin(offshore_parks_diss, how="left", predicate='within')
offshore_geo_parks_grouped = pandas.DataFrame(offshore_geo_parks.drop(columns='geometry'))

# set Nordsee or Ostsee to 0 or 1, deppending on 
offshore_geo_parks_grouped[["ClusterNordsee","ClusterOstsee"]] = offshore_geo_parks_grouped[["ClusterNordsee","ClusterOstsee"]].notnull().astype(int)

# Groupby Windparkname
offshore_geo_parks_grouped = offshore_geo_parks_grouped.groupby("Windparkname").agg({
    "ClusterNordsee":["mean"],
    "ClusterOstsee":["mean"],
    "Anlagenzahl":["sum"],
    "Nettonennleistung":["sum"],
    "Bruttoleistung":["sum"],
    "SpezifischeLeistung":["mean"],
    "Nabenhoehe":["mean"],
    "Rotordurchmesser":["mean"],
    "Typenbezeichnung": lambda x:x.value_counts().index[0]
})

offshore_geo_parks_grouped.round(1)

,ClusterNordsee,ClusterOstsee,Anlagenzahl,Nettonennleistung,Bruttoleistung,SpezifischeLeistung,Nabenhoehe,Rotordurchmesser,Typenbezeichnung
,mean,mean,sum,sum,sum,mean,mean,mean,<lambda>
Windparkname,,,,,,,,,
Albatros,1.0,0.0,16,117600.0,117600.0,394.6,105.0,154.0,SWT 7.0 154
Amrumbank West,1.0,0.0,79,298620.0,298620.0,336.1,90.0,119.7,SWT-3.6-120
Arkona-Becken Südost,0.0,1.0,59,371700.0,371700.0,338.2,102.0,154.0,Siemens 6.0-154 PB-HWRT
BARD Offshore 1,1.0,0.0,80,400000.0,400000.0,427.7,90.0,122.0,BARD 5.0
Borkum Riffgrund 1,1.0,0.0,78,312000.0,312000.0,353.7,87.0,120.0,SWT-3.6-120 V3
Borkum Riffgrund 2,1.0,0.0,56,464800.0,464800.0,392.9,112.2,164.0,MHI Vestas Offshore Wind V164-8.0
Butendiek,1.0,0.0,80,288000.0,288000.0,318.3,91.0,120.0,SWT-3.6-120
DanTysk,1.0,0.0,78,294840.0,294840.0,334.2,88.0,120.0,SWT-3.6-120
